In [35]:
import torch
# print(torch.__version__)
# !pip install torch-scatter torch-sparse -f https://data.pyg.org/whl/torch-{torch.__version__}.html
# !pip install torch-geometric
%load_ext autoreload
%autoreload 2

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

from networkx.generators.random_graphs import erdos_renyi_graph
from networkx.generators.random_graphs import barabasi_albert_graph
from networkx.generators.community import stochastic_block_model
from networkx.generators.random_graphs import watts_strogatz_graph
from networkx.generators.community import random_partition_graph

import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import math
from tqdm import tqdm
import seaborn as sns
from sklearn.decomposition import FactorAnalysis

import random
import numpy
import torch
%load_ext autoreload
%autoreload 2

%matplotlib inline
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))



from networkx.generators.random_graphs import erdos_renyi_graph
from networkx.generators.random_graphs import barabasi_albert_graph
from networkx.generators.community import stochastic_block_model
from networkx.generators.random_graphs import watts_strogatz_graph
from networkx.generators.community import random_partition_graph
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt

import math
from tqdm import tqdm
import seaborn as sns
from sklearn.decomposition import FactorAnalysis
import random
import os
import collections
from random import sample
from scipy.sparse import csr_matrix
from scipy.sparse import csgraph
from scipy.sparse.linalg import inv



import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.sparse as sp
import torch
from torch import Tensor
import torch_geometric
from torch_geometric.utils import to_networkx
from torch_geometric.datasets import Planetoid
import networkx as nx
from networkx.algorithms import community
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data_dir = "./data"
os.makedirs(data_dir, exist_ok=True)
import os
os.getcwd()
dataset = os.path.join(os.getcwd(),'Citeseer')
from torch_geometric.datasets import Planetoid
from torch_geometric.utils import to_dense_adj
from torch_geometric.nn import GCNConv, BatchNorm, GraphConv



dataset= Planetoid(root=dataset, name='Citeseer')
print(dataset[0])
adj = to_dense_adj(dataset[0].edge_index)
adj = adj[0]
labels = dataset[0].y
labels = labels.numpy()

X = dataset[0].x
X = X.to_dense()
N = X.shape[0]
NO_OF_CLASSES =  len(set(np.array(dataset[0].y)))

print(X.shape, adj.shape)

nn = int(1*N)
X = X[:nn,:]
adj = adj[:nn,:nn]
labels = labels[:nn]
print(X.shape,adj.shape)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/var/folders/kr/y756xq8918q0kf1g4nk4vykh0000gn/T/ipykernel_72598/560116527.py:8: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/var/folders/kr/y756xq8918q0kf1g4nk4vykh0000gn/T/ipykernel_72598/560116527.py:34: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


Data(x=[3327, 3703], edge_index=[2, 9104], y=[3327], train_mask=[3327], val_mask=[3327], test_mask=[3327])
torch.Size([3327, 3703]) torch.Size([3327, 3327])
torch.Size([3327, 3703]) torch.Size([3327, 3327])


In [36]:
Data = dataset[0]

In [37]:
Data

Data(x=[3327, 3703], edge_index=[2, 9104], y=[3327], train_mask=[3327], val_mask=[3327], test_mask=[3327])

In [38]:
import torch_geometric.transforms as T

split = T.RandomLinkSplit(
    num_val=0.01,
    num_test=0.005,
    is_undirected=True,
    add_negative_train_samples=False,
    neg_sampling_ratio=1.0,
)
tra, va, tes = split(Data)

In [39]:
tes

Data(x=[3327, 3703], edge_index=[2, 9060], y=[3327], train_mask=[3327], val_mask=[3327], test_mask=[3327], edge_label=[44], edge_label_index=[2, 44])

In [40]:
def get_laplacian(adj):
    b=torch.ones(adj.shape[0])
    return torch.diag(adj@b)-adj

theta = get_laplacian(adj)
print(theta.shape)

torch.Size([3327, 3327])


In [41]:
features = X.numpy()
NO_OF_NODES = X.shape[0]
def convertScipyToTensor(coo):
  try:
    coo = coo.tocoo()
  except:
    coo = coo
  values = coo.data
  indices = np.vstack((coo.row, coo.col))

  i = torch.LongTensor(indices)
  v = torch.FloatTensor(values)
  shape = coo.shape

  return torch.sparse.FloatTensor(i, v, torch.Size(shape))

print(NO_OF_CLASSES,NO_OF_NODES)

6 3327


In [42]:
from scipy.sparse import random
from scipy.sparse.linalg import norm
from scipy.sparse import csr_matrix

p = X.shape[0]
k = int(p*0.5)
n = X.shape[1]
lambda_param = 100
beta_param = 50
alpha_param = 100
gamma_param = 100
lr = 1e-5
thresh = 1e-10

from scipy.sparse import random
from scipy.stats import rv_continuous
class CustomDistribution(rv_continuous):
    def _rvs(self,  size=None, random_state=None):
        return random_state.standard_normal(size)
temp = CustomDistribution(seed=1)
temp2 = temp()  # get a frozen version of the distribution
X_tilde = random(k, n, density=0.25, random_state=1, data_rvs=temp2.rvs)
C = random(p, k, density=0.25, random_state=1, data_rvs=temp2.rvs)

In [43]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from random import sample

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(X.shape[1], 64)
        self.conv2 = GCNConv(64, NO_OF_CLASSES)

    def reset_parameters(self):
        self.conv1.reset_parameters()
        self.conv2.reset_parameters()

    def forward(self, x, edge_index):

        #print("Checking 1: x", x.shape, "Edge index:", edge_index.shape)
        x = self.conv1(x, edge_index)
        #print("Checking 2: convolution done, new x:", x.shape)
        x = F.relu(x)
        #print("Checking 3: x", x.shape, "training:", self.training)
        x = F.dropout(x, training=self.training)
        #print("Checking 4: dropout done new x", x.shape, "Edge index:", edge_index.shape)
        x = self.conv2(x, edge_index)
        #print("Checking 5: x", x.shape)

        return F.log_softmax(x, dim=1)

In [44]:
def experiment_sparsity(lambda_param,beta_param,gamma_param,C,theta,X):
      p = X.shape[0]
      k = int(p*0.5)
      n = X.shape[1]
      ones = csr_matrix(np.ones((k,k)))
      ones = convertScipyToTensor(ones)
      ones = ones.to_dense()
      J = np.outer(np.ones(k), np.ones(k))/k
      J = csr_matrix(J)
      J = convertScipyToTensor(J)
      J = J.to_dense()
      zeros = csr_matrix(np.zeros((p,k)))
      zeros = convertScipyToTensor(zeros)
      zeros = zeros.to_dense()
#       X_tilde = convertScipyToTensor(X_tilde)
#       X_tilde = X_tilde.to_dense()
      C = convertScipyToTensor(C)
      C = C.to_dense()
      eye = torch.eye(k)
      try:
        theta = convertScipyToTensor(theta)
      except:
        theta = theta
      try:
        X = convertScipyToTensor(X)
        X = X.to_dense()
      except:
        X = X

      if(torch.cuda.is_available()):
        print("yes")
#         X_tilde = X_tilde.cuda()
        C = C.cuda()
        theta = theta.cuda()
        X = X.cuda()
        J = J.cuda()
        zeros = zeros.cuda()
        ones = ones.cuda()
        eye = eye.cuda()

      def update(C,i):
          global L
          thetaC = theta@C
          CT = torch.transpose(C,0,1)
          t1 = CT@thetaC + J
          term_bracket = torch.linalg.pinv(t1)
 
 #           X_tildeT = torch.transpose(X_tilde,0,1)#NOT needed 21Mar
 #           CX_tilde = C@X_tilde  #NOT needed 21Mar
 #           thetacX_tilde = thetaC@(X_tilde) #NOT needed 21M
          
          L = 1/k

          t1 = -2*gamma_param*(thetaC@term_bracket)
#           t2 = alpha_param*(CX_tilde-X)@(X_tildeT)   #NOT needed 21
#           t3 = 2*thetacX_tilde@(X_tildeT)   #NOT needed 21
          t4 = lambda_param*(C@ones)
          t5 = 2*beta_param*(thetaC@CT@thetaC)
#           T2 = (t1+t2+t3+t4+t5)/L
          T2=(t1+t4+t5)/L
          Cnew = (C-T2).maximum(zeros)
#           t1 = CT@thetaC*(2/alpha_param)  #NOT needed 21
#           t2 = CT@C#NOT needed 21
#           t1 = torch.linalg.pinv(t1+t2)#NOT needed 21
#           t1 = t1@CT#NOT needed 21
#           t1 = t1@X#NOT needed 21
#           X_tilde_new = t1#NOT needed 21
          Cnew[Cnew<thresh] = thresh
          for i in range(len(Cnew)):
              Cnew[i] = Cnew[i]/torch.linalg.norm(Cnew[i],1)
#           for i in range(len(X_tilde_new)):
#             X_tilde_new[i] = X_tilde_new[i]/torch.linalg.norm(X_tilde_new[i],1)
#           return X_tilde_new,Cnew  #ret cnew
          return Cnew

      for i in tqdm(range(20)):   #update C only 21
          C = update(C,i)
    

      return C

In [45]:
def get_accuracy(C_0,L):
    global labels, NO_OF_CLASSES,k
    t=[]
    for i in [1,2,3,4,5,6,7,8,9,10]: 
        C_0_new=np.zeros(C_0.shape)
        for i in range(C_0.shape[0]):
            C_0_new[i][np.argmax(C_0[i])]=1
        # print(C_0_new)
        # C_0_new=C_0
        from scipy import sparse
        #Lc=C_0.T@L@C_0
        Lc=C_0_new.T@L@C_0_new
        # print("L:", Lc.shape)
        # Lc=L_new
        #print(Lc)
        Wc=(-1*Lc)*(1-np.eye(Lc.shape[0]))
        # print("W:", Wc.shape)
        Wc[Wc<0.1]=0
        Wc=sparse.csr_matrix(Wc)
        Wc = Wc.tocoo()
        row = torch.from_numpy(Wc.row).to(torch.long)
        col = torch.from_numpy(Wc.col).to(torch.long)
        edge_index_coarsen2 = torch.stack([row, col], dim=0)
        #print("edgecoarsen:", edge_index_coarsen2.shape)
        edge_weight = torch.from_numpy(Wc.data)
        #print("edgeweight:", edge_weight.shape)
        def one_hot(x, class_count):
            return torch.eye(class_count)[x, :]

        device = torch.device('cpu')
        labels=labels
        Y = labels
        #print("Y:", Y.shape)
        Y = one_hot(Y,NO_OF_CLASSES)
        # NO_OF_CLASSES=Y.shape[1]
        P=np.linalg.pinv(C_0_new)
        labels_coarse = torch.argmax(torch.sparse.mm(torch.Tensor(P).double() , Y.double()).double() , 1)
        #print("Lables:", labels_coarse.shape)

        #torch.Tensor(C2)@X
        Wc=Wc.toarray()
        #Wc[Wc<0.01]=0
        C2=np.linalg.pinv(C_0_new)
        model=Net().to(device)
        device = torch.device('cpu')
        lr=0.01
        decay=0.0001
        try:
          X=np.array(features.todense())
        except:
          X = np.array(features)
        #print("X:",X.shape)
        optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=decay)
        # criterion=torch.nn.CrossEntropyLoss()
        x=sample(range(0, int(k)), k)
      
        from datetime import datetime
        Xt=P@X
        # Xt=X_t_0
        def train():
            model.train()
            optimizer.zero_grad()
            out = model(torch.Tensor(Xt).to(device),edge_index_coarsen2)
            loss = F.nll_loss(out[x], labels_coarse[x])
            loss.backward()
            optimizer.step()
            return loss
        now1 = datetime.now()
        losses=[]
        for epoch in range(60):
            loss=train()
            losses.append(loss)
            if(epoch%100==0):
                print(f'Epoch: {epoch:03d},loss: {loss:.4f}')
        now2 = datetime.now()        
        pred=model(torch.Tensor(Xt).to(device),edge_index_coarsen2).argmax(dim=1)        
        def train_accuracy():
            model.eval()
            correct = (pred[x] == labels_coarse[x]).sum()
            acc = int(correct) /len(x)
            return acc
    
        t+=[(now2-now1).total_seconds()]

        zz=sample(range(0, int(NO_OF_NODES)), NO_OF_NODES)
        Wc=sparse.csr_matrix(adj)
        Wc = Wc.tocoo()
        row = torch.from_numpy(Wc.row).to(torch.long)
        col = torch.from_numpy(Wc.col).to(torch.long)
        edge_index_coarsen = torch.stack([row, col], dim=0)
        edge_weight = torch.from_numpy(Wc.data)
        pred=model(torch.Tensor(X),edge_index_coarsen).argmax(dim=1)
        pred=np.array(pred)
        correct =(pred[zz]==labels[zz]).sum()
        acc = int(correct) /NO_OF_NODES
        return acc


In [46]:
highest_accuracy=0
C = random(p, k, density=0.15, random_state=1, data_rvs=temp2.rvs)
C_0 = experiment_sparsity(100,0.1,100,C,theta,X)

             

100%|███████████████████████████████████████████| 20/20 [00:12<00:00,  1.58it/s]


In [14]:
# highest_accuracy=0
# for lambda_param in [100,10,1,0.1,0.01]:
#   for beta_param in [100,10,1,0.1,0.01,0.001]:
#       for gamma_param in [100,10,1,0.1,0.01]:

#         av = []

#         for _ in range(2):
#             avg_accuracy_all=[]
#             for _ in range(1):
#               C = random(p, k, density=0.15, random_state=1, data_rvs=temp2.rvs)
#               C_0 = experiment_sparsity(lambda_param,beta_param,gamma_param,C,theta,X)
#               L = theta
#               C_0 = C_0.cpu().detach().numpy()
#               C_t_0 = C_0.T
#               try:
#                 L = L.cpu().detach().numpy()
#               except:
#                 L = L

#               acc = get_accuracy(C_0,L)
#               av.append(acc)
#               if highest_accuracy<acc:
#                 highest_accuracy=acc
#                 print("Accuracy = " + str(acc) + " " + str(alpha_param)+" " + str(beta_param)+" "+str(gamma_param))
#         print("Average accuracy = " + str(np.mean(av)*100)  + " +/- " + str(np.std(av)*100))
#         print("Params =  " + str(lambda_param)+" " + str(beta_param)+" "+str(gamma_param))

100%|███████████████████████████████████████████| 20/20 [00:12<00:00,  1.65it/s]


Epoch: 000,loss: 1.7922
Accuracy = 0.3432521791403667 100 100 100


100%|███████████████████████████████████████████| 20/20 [00:12<00:00,  1.64it/s]


Epoch: 000,loss: 20059523072.0000
Average accuracy = 26.75082657048392 +/- 7.574391343552751
Params =  100 100 100


100%|███████████████████████████████████████████| 20/20 [00:11<00:00,  1.69it/s]


...

Epoch: 000,loss: 1.7933


100%|███████████████████████████████████████████| 20/20 [00:12<00:00,  1.58it/s]


Epoch: 000,loss: 1.7952
Average accuracy = 71.8214607754734 +/- 0.5861136158701541
Params =  10 0.001 10


 75%|████████████████████████████████▎          | 15/20 [00:09<00:03,  1.56it/s]


KeyboardInterrupt: 

In [47]:

C_0_new=C_0
from scipy import sparse
#Lc=C_0.T@L@C_0
Lc=C_0_new.T@theta@C_0_new
print("L:", Lc.shape)
# Lc=L_new
#print(Lc)
Wc=(-1*Lc)*(1-np.eye(Lc.shape[0]))
# print("W:", Wc.shape)
Wc[Wc<0.1]=0
Wc=sparse.csr_matrix(Wc)
Wc = Wc.tocoo()
row = torch.from_numpy(Wc.row).to(torch.long)
col = torch.from_numpy(Wc.col).to(torch.long)
edge_index_coarsen2 = torch.stack([row, col], dim=0)
print("edgecoarsen:", edge_index_coarsen2.shape)
edge_weight = torch.from_numpy(Wc.data)
#print("edgeweight:", edge_weight.shape)
def one_hot(x, class_count):
    return torch.eye(class_count)[x, :]

device = torch.device('cpu')
labels=labels
Y = labels
#print("Y:", Y.shape)
Y = one_hot(Y,NO_OF_CLASSES)
# NO_OF_CLASSES=Y.shape[1]
P=np.linalg.pinv(C_0_new)
labels_coarse = torch.argmax(torch.sparse.mm(torch.Tensor(P).double() , Y.double()).double() , 1)
print("Lables:", labels_coarse.shape)


L: torch.Size([1663, 1663])
edgecoarsen: torch.Size([2, 2728])
Lables: torch.Size([1663])


In [48]:
P = torch.Tensor(P)
Xt=P@X

In [49]:
Data

Data(x=[3327, 3703], edge_index=[2, 9104], y=[3327], train_mask=[3327], val_mask=[3327], test_mask=[3327])

In [50]:
Data.x = Xt
Data.edge_index = edge_index_coarsen2
Data.y=labels_coarse
Data.train_mask =0
Data.test_mask = 0
Data.val_mask =0
Data

Data(x=[1663, 3703], edge_index=[2, 2728], y=[1663], train_mask=0, val_mask=0, test_mask=0)

In [51]:
graph =Data
graph

Data(x=[1663, 3703], edge_index=[2, 2728], y=[1663], train_mask=0, val_mask=0, test_mask=0)

## Citeseer Original data Link prediction AUC = 0.874

## Cora Original data Link prediction AUC =0.886

In [52]:
import torch
import torchvision
from sklearn.metrics import roc_auc_score
from torch_geometric.utils import negative_sampling


class Net(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def encode(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv2(x, edge_index)

    def decode(self, z, edge_label_index):
        return (z[edge_label_index[0]] * z[edge_label_index[1]]).sum(
            dim=-1
        )  # product of a pair of nodes on each edge

    def decode_all(self, z):
        prob_adj = z @ z.t()
        return (prob_adj > 0).nonzero(as_tuple=False).t()


def train_link_predictor(
    model, train_data, val_data, optimizer, criterion, n_epochs=400
):

    for epoch in range(1, n_epochs + 1):

        model.train()
        optimizer.zero_grad()
        z = model.encode(train_data.x, train_data.edge_index)

        # sampling training negatives for every training epoch
        neg_edge_index = negative_sampling(
            edge_index=train_data.edge_index, num_nodes=train_data.num_nodes,
            num_neg_samples=train_data.edge_label_index.size(1), method='sparse')

        edge_label_index = torch.cat(
            [train_data.edge_label_index, neg_edge_index],
            dim=-1,
        )
        edge_label = torch.cat([
            train_data.edge_label,
            train_data.edge_label.new_zeros(neg_edge_index.size(1))
        ], dim=0)

        out = model.decode(z, edge_label_index).view(-1)
        loss = criterion(out, edge_label)
        loss.backward()
        optimizer.step()

        val_auc = eval_link_predictor(model, val_data)

        if epoch % 100 == 0:
            print(f"Epoch: {epoch:03d}, Train Loss: {loss:.3f}, Val AUC: {val_auc:.3f}")

    return model


@torch.no_grad()
def eval_link_predictor(model, data):

    model.eval()
    z = model.encode(data.x, data.edge_index)
    out = model.decode(z, data.edge_label_index).view(-1).sigmoid()

    return roc_auc_score(data.edge_label.cpu().numpy(), out.cpu().numpy())

In [53]:
import torch_geometric.transforms as T

split = T.RandomLinkSplit(
    num_val=0.1,
    num_test=0.1,
    is_undirected=True,
    add_negative_train_samples=False,
    neg_sampling_ratio=1.0,
)
train_data, val_data, test_data = split(graph)
train_data = train_data.cpu()
test_data = test_data.cpu()
val_data = val_data.cpu()


In [54]:
train_data

Data(x=[1663, 3703], edge_index=[2, 2184], y=[1663], train_mask=0, val_mask=0, test_mask=0, edge_label=[1092], edge_label_index=[2, 1092])

In [55]:
# test_data =tes.cpu()
# tes

In [56]:
model = Net(dataset.num_features, 128, 64)
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()
model = train_link_predictor(model, train_data, val_data, optimizer, criterion)

test_auc = eval_link_predictor(model, test_data)
print(f"Test: {test_auc:.3f}")

Epoch: 100, Train Loss: 239978305340905984360448000.000, Val AUC: 0.662
Epoch: 200, Train Loss: 298881452392895344264871936.000, Val AUC: 0.662
Epoch: 300, Train Loss: 257072294331458193427791872.000, Val AUC: 0.662
Epoch: 400, Train Loss: 384535219150350905283444736.000, Val AUC: 0.662
Test: 0.669
